# Projecting Insulin usage and Blood Glucose usage over time

This notebook takes the data from the `data.csv` file in this same directory and then runs the [Granite TimeSeries](URL) model against it to project insulin usage and what the CGMS Sensor has. This data is from JJ Asghar pulled from his Medtronic Pump and Sensor, and approves using this for this exploration of information and learning.

## Set up
Run the following cell to set up the environment.

In [ ]:
# Install pandas because well pandas is important
! pip install pandas
# Install the tsfm library
! pip install "tsfm_public[notebooks] @ git+https://github.com/ibm-granite/granite-tsfm.git@v0.2.17" -U

# Import the modules you'll need for this notebook
import pandas as pd
import matplotlib.pyplot as plt

from tsfm_public import (
    TimeSeriesForecastingPipeline,
    TinyTimeMixerForPrediction,
)
from tsfm_public.toolkit.visualization import plot_predictions

timestamp_column = "Date_Time"
target_column = ["Sensor Glucose (mg/dL)"]
context_length = 1536

DATA_FILE_PATH = "data.csv"
df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)

In [ ]:
# Sanity check for the glucose sensor
DATA_FILE_PATH = "data.csv"
df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)
df_glucose_sensor = df.iloc[2689:21338]
df_glucose_sensor.loc[:,["Date_Time","Sensor Glucose (mg/dL)"]]

In [ ]:
# Sanity check for daily basal rate
DATA_FILE_PATH = "data.csv"
df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)
df_daily_basal_rate = df.iloc[2620:2684]
df_daily_basal_rate.loc[:,["Date_Time","Bolus Volume Delivered (U)"]]

## Output the general graph of the Sensor data

In [ ]:
DATA_FILE_PATH = "data.csv"
df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)
df = df.iloc[2689:5000,:]

# Fill NA/NaN values by propagating the last valid value.
df = df.ffill()

df = df.loc[:,["Date_Time","Sensor Glucose (mg/dL)"]]
df.dropna()

timestamp_column = ["Date_Time"]
target_column = ["Sensor Glucose (mg/dL)"]

df[target_column] = df[target_column].astype(float)
df["Date_Time"] = pd.to_datetime(df["Date_Time"])
df = df.sort_values("Date_Time")
df.plot(x="Date_Time", y="Sensor Glucose (mg/dL)", figsize=(20, 10))

## Project the Sensor data via the model

In [ ]:
DATA_FILE_PATH = "data.csv"
input_df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)
input_df = input_df.iloc[2689:5000,:]

# Fill NA/NaN values by propagating the last valid value.
input_df = input_df.ffill()

input_df = input_df.loc[:,["Date_Time","Sensor Glucose (mg/dL)"]]

timestamp_column = ["Date_Time"]
target_column = ["Sensor Glucose (mg/dL)"]

input_df[target_column] = input_df[target_column].astype(float)
input_df["Date_Time"] = pd.to_datetime(input_df["Date_Time"])

input_df = input_df.sort_values("Date_Time")
input_df.plot(x="Date_Time", y="Sensor Glucose (mg/dL)", figsize=(20, 10))

zeroshot_model = TinyTimeMixerForPrediction.from_pretrained(
    "ibm-granite/granite-timeseries-ttm-r2",  # Name of the model on Hugging Face
    num_input_channels=len(target_column),  # tsp.num_input_channels
)

# Create a pipeline.
pipeline = TimeSeriesForecastingPipeline(
    zeroshot_model,
    timestamp_column="Date_Time",
    id_columns=[],
    target_columns=target_column,
    explode_forecasts=False,
    freq="5m",
    device="cpu",  # Specify your local GPU or CPU.
)

# Make a forecast on the target column given the input data.
zeroshot_forecast = pipeline(input_df)

# Plot the historical data and predicted series.
plot_predictions(
    input_df=input_df,
    predictions_df=zeroshot_forecast,
    freq="h",
    timestamp_column="Date_Time",
    channel="Sensor Glucose (mg/dL)",
    indices=[-1],
    num_plots=1,
)

## Project the Bolus Volume over time, with smoothing out the data

In [ ]:
DATA_FILE_PATH = "data.csv"
input_df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)
input_df = input_df.iloc[2:2615,:]

# Fill NA/NaN values by propagating the last valid value.
#input_df = input_df.fillna(0)
input_df = input_df.ffill()

input_df = input_df.loc[:,["Date_Time","Bolus Volume Delivered (U)"]]
#input_df.dropna()

timestamp_column = ["Date_Time"]
input_df["smooth_bolus"] = input_df["Bolus Volume Delivered (U)"].rolling(window=3).mean()
target_column = ["smooth_bolus"]

input_df[target_column] = input_df[target_column].astype(float)
input_df["Date_Time"] = pd.to_datetime(input_df["Date_Time"])

input_df = input_df.sort_values("Date_Time")
input_df.plot(x="Date_Time", y="smooth_bolus", figsize=(20, 10))

zeroshot_model = TinyTimeMixerForPrediction.from_pretrained(
    "ibm-granite/granite-timeseries-ttm-r2",  # Name of the model on Hugging Face
    num_input_channels=len(target_column),  # tsp.num_input_channels
)

# Create a pipeline.
pipeline = TimeSeriesForecastingPipeline(
    zeroshot_model,
    timestamp_column="Date_Time",
    id_columns=[],
    target_columns=target_column,
    explode_forecasts=False,
    freq="5m",
    device="cpu",  # Specify your local GPU or CPU.
)

# Make a forecast on the target column given the input data.
zeroshot_forecast = pipeline(input_df)

# Plot the historical data and predicted series.
plot_predictions(
    input_df=input_df,
    predictions_df=zeroshot_forecast,
    freq="h",
    timestamp_column="Date_Time",
    channel="smooth_bolus",
    indices=[-1],
    num_plots=1,
)

## Project the Bolus Volume over time, without smoothing out the data

In [ ]:
DATA_FILE_PATH = "data.csv"
input_df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)
input_df = input_df.iloc[2620:2684,:]

# Fill NA/NaN values by propagating the last valid value.
input_df = input_df.ffill()

input_df = input_df.loc[:,["Date_Time","Bolus Volume Delivered (U)"]]
    
timestamp_column = ["Date_Time"]
target_column = ["Bolus Volume Delivered (U)"]

input_df[target_column] = input_df[target_column].astype(float)
input_df["Date_Time"] = pd.to_datetime(input_df["Date_Time"])

input_df = input_df.sort_values("Date_Time")
input_df.plot(x="Date_Time", y="Bolus Volume Delivered (U)", figsize=(20, 10))

zeroshot_model = TinyTimeMixerForPrediction.from_pretrained(
    "ibm-granite/granite-timeseries-ttm-r2",  # Name of the model on Hugging Face
    num_input_channels=len(target_column),  # tsp.num_input_channels
)

# Create a pipeline.
pipeline = TimeSeriesForecastingPipeline(
    zeroshot_model,
    timestamp_column="Date_Time",
    id_columns=[],
    target_columns=target_column,
    explode_forecasts=False,
    freq="d",
    device="cpu",  # Specify your local GPU or CPU.
)

# Make a forecast on the target column given the input data.
zeroshot_forecast = pipeline(input_df)

# Plot the historical data and predicted series.
plot_predictions(
    input_df=input_df,
    predictions_df=zeroshot_forecast,
    freq="d",
    timestamp_column="Date_Time",
    channel="Bolus Volume Delivered (U)",
    indices=[-1],
    num_plots=1,
)